In [75]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from requests import get
from bs4 import BeautifulSoup
import lxml

function source: https://github.com/vishaalagartha/basketball_reference_scraper/blob/master/basketball_reference_scraper/seasons.py

In [2]:
def get_standings(date=None):
    if date is None:
        date = datetime.now()
    else:
        date = pd.to_datetime(date)
    d = {}
    r = get(f'https://www.basketball-reference.com/friv/standings.fcgi?month={date.month}&day={date.day}&year={date.year}')
    if r.status_code==200:
        soup = BeautifulSoup(r.content, 'html.parser')
        e_table = soup.find('table', attrs={'id': 'standings_e'})
        e_df = pd.read_html(str(e_table))[0]
        w_table = soup.find('table', attrs={'id': 'standings_w'})
        w_df = pd.read_html(str(w_table))[0]
        e_df.rename(columns={'Eastern Conference': 'TEAM'}, inplace=True)
        w_df.rename(columns={'Western Conference': 'TEAM'}, inplace=True)
        d['EASTERN_CONF'] = e_df
        d['WESTERN_CONF'] = w_df
    return d

In [90]:
df = pd.DataFrame(columns=['TEAM','W','L','W/L%','GB','PW','PL','PS/G','PA/G','year'])
years = []
for i in range(2000,2020):
    years.append(str(i)+'-06-30')  
    for y in years:
        p = get_standings(date=y)
        cat = pd.concat([p['EASTERN_CONF'],p['WESTERN_CONF']])
        cat['year'] = str(i)
        df = df.append(cat,ignore_index = True)
        

In [91]:
standings_df = df.drop(df[df['TEAM'].str.contains('Division')].index.values,axis = 0)

In [94]:
standings_df.to_csv('Standings.csv',index=False)